In [10]:
import pandas as pd
import requests
from datetime import datetime


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)


### FUNÇÃO PARA PEGAR TOKEN DE AUTENTICAÇÃO NA API
def get_token(email,senha):
    body = {"email": email,"password": senha}
    r = requests.post('https://api.oplab.com.br/v3/domain/users/authenticate',json=body)
    return r.json()['access-token']

### FUNÇÃO PARA COLETAR LISTA DE AÇÕES COM MAIOR LIQUIDEZ EM OPÇÕES
def pegar_mais_liqs(token,rank = 20):
    lista = []
    r = requests.get('https://api.oplab.com.br/v3/market/statistics/realtime/highest_options_volume?order_by=put&limit={}'.format(rank),headers={"Access-Token": token}).json()
    for i in r:
        lista.append(i['symbol'])
    return lista

### FUNÇÃO PARA COLETAR TODAS AS OPÇÕES DISPONÍVEIS POR ATIVO
def get_all_options(symbol,token):
    r = requests.get('https://api.oplab.com.br/v3/market/options/{}'.format(symbol), headers={"Access-Token": token})
    return r.json()

### FUNÇÃO PARA ORGANIZAR OPÇÕES EM UM DATAFRAME
def ofertas_opcoes(ativo,token,tamanho_contrato_dif = 1,ask_dif_zero = 1):
    symbol = []
    strike = []
    bid = []
    ask = []
    moneyness = []
    ativo_obj = []
    tipo = []
    vcto = []
    spot_price = []
    contract_size = []
    all_options = get_all_options(ativo,token)
    for i in range(len(all_options)):
        maturity = (datetime.strptime(all_options[i]['due_date'],"%Y-%m-%d")-datetime.today())
        if all_options[i]['strike'] >0 and (all_options[i]['contract_size'] < 100) == tamanho_contrato_dif and (all_options[i]['ask'] > 0) == ask_dif_zero:
            p_spot = 0 if all_options[i]['spot_price'] == 0 else all_options[i]['spot_price']
            mnns = 0 if all_options[i]['spot_price'] == 0 else all_options[i]['strike'] / all_options[i]['spot_price'] -1 
            symbol.append(all_options[i]['symbol'])
            tipo.append(all_options[i]['type'])
            strike.append(all_options[i]['strike'])
            bid.append(all_options[i]['bid'])
            ask.append(all_options[i]['ask'])
            moneyness.append(round(mnns*100,4))
            ativo_obj.append(ativo)
            vcto.append(maturity.days)
            spot_price.append(p_spot)
            contract_size.append(all_options[i]['contract_size'])

    ofertas = pd.DataFrame({'OBJ':ativo_obj,'OPÇÃO':symbol,'TIPO':tipo,'SPOT':spot_price,'K':strike,'BID':bid,'ASK':ask,'DIST_ATM':moneyness,'VCTO':vcto,'SIZE':contract_size})
    return ofertas

### INSERIR EMAIL E SENHA --> get_token('seu@email.com','sua_senha')
try:
    token = get_token() 
except:
    print('TOKEN ERRADO')

### COLETAR LISTA DE 20 ATIVOS COM MAIOR LIQUIDEZ EM OPÇÕES
lista_ativos = pegar_mais_liqs(token,20)

### FILTRAR OPÇÕES COM CONTRACT_SIZE DIFERENTE DE 100
for ativo in lista_ativos:
    ### SE O ATIVO FOR DIFERENTE DE IBOV
    if ativo == 'IBOV':
        continue
    ofertas = ofertas_opcoes(ativo,token,tamanho_contrato_dif = 1,ask_dif_zero = 1)
    if len(ofertas) > 0:
        print(ofertas)
    else:
        continue

#

      OBJ     OPÇÃO  TIPO   SPOT     K  BID    ASK  DIST_ATM  VCTO  SIZE
0   PETR4  PETRM190   PUT   0.00  0.24    0   0.01    0.0000    41     1
1   PETR4  PETRM170   PUT   0.00  0.77    0   0.01    0.0000    41     1
2   PETR4  PETRM200   PUT   0.00  1.12    0   0.01    0.0000    41     1
3   PETR4  PETRM140   PUT   0.00  1.39    0   0.01    0.0000    41     1
4   PETR4  PETRM180   PUT   0.00  1.59    0   0.01    0.0000    41     1
5   PETR4  PETRM210   PUT  24.82  1.99    0   0.01  -91.9823    41     1
6   PETR4  PETRM150   PUT   0.00  2.11    0   0.01    0.0000    41     1
7   PETR4  PETRM160   PUT   0.00  2.83    0   0.01    0.0000    41     1
8   PETR4  PETRX150   PUT   0.00  1.55    0   0.01    0.0000   370     1
9   PETR4  PETRD120  CALL  24.74  1.55    0  28.75  -93.7348   496     1
10  PETR4  PETRP120   PUT   0.00  1.55    0   0.05    0.0000   496     1
11  PETR4  PETRD100  CALL  24.74  1.99    0  27.73  -91.9563   496     1
12  PETR4  PETRP100   PUT  24.75  1.99    0   0.20 